In [7]:
import re 

def process_log(log_path):
    log = dict()
    test = 0
    train = 0
    log["file_name"] = log_path.split("/")[-1]
    log["max_train_iter"] = 0
    
    lines = []
    with open(log_path, 'r') as log_file:
        
        for line in log_file:
            lines.append(line)
            if "testing on test data" in line:
                log[f"test_test_{test}"] = dict()
                updating = f"test_test_{test}"
                log[updating]["rewards"] = [] 
                test += 1
            elif "testing on train data" in line:
                log[f"test_train_{train}"] = dict()
                updating = f"test_train_{train}"
                log[updating]["rewards"] = []
                train += 1
            elif "training iteration" in line:
                iteration = int(line.split()[-1])
                log[f"train_{iteration}"] = dict()
                updating = f"train_{iteration}"
                
                if log["max_train_iter"] < iteration:
                    log["max_train_iter"] = iteration
                    
                log[updating]["rewards"] = []
            elif "success" in line:
                info = re.findall(r"\d+.\d+|[+-]*\d+", line)
                log[updating]["success"] = int(info[0])
                log[updating]["total"] = int(info[1])
                log[updating]["ave_loss"]= float(info[2])
            elif "[tensor" in line:
                try:
                    rewards = re.findall(r"\d+.\d+|[+-]*\d+", line)
                    log[updating]["rewards"].append([float(reward) for reward in rewards])
                except UnboundLocalError:
                    print(len(lines))
                    #print (lines[-1])
                    return
            elif " in " in line:
                log[updating]["time_spent"] = float(line.split()[-1])
            
    return log

def get_x_y(log):
    x = range(log["max_train_iter"]+1)
    y = []
    for ct in range(log["max_train_iter"]+1):
        # calculate average reward
        ave_roll_out = [sum(roll_out_reward)/len(roll_out_reward) for roll_out_reward in log[f"train_{ct}"]["rewards"]]
        ave_reward = sum(ave_roll_out)/len(ave_roll_out)
        y.append(ave_reward)
    return x, y

def plot_training_rewards(log_path):
    log = process_log(log_path) 
    x, y = get_x_y(log)
    fig, ax = plt.subplots()
    ax.plot(x, y)
    
    ax.set(xlabel='iteration', ylabel='reward',
       title=log["file_name"])
    ax.grid()
    plt.show()


In [8]:
import os
data_dir = os.path.abspath(os.path.join(os.path.abspath("__file__"), "../../data" ))
log_dir = os.path.join(data_dir, "log/")
# log_files = [ log_dir + filename for filename in os.listdir(log_dir)]
filename = "img_test_1_prob_things_all_certain_lr_0.001.log"
log_files = [log_dir + filename]

In [9]:
for log_file_path in log_files:
    plot_training_rewards(log_file_path)

ValueError: could not convert string to float: '0]]'

In [2]:
def prob_to_cat(prob, offsets):
        for ct, offset in enumerate(offsets):
            if prob > offset:
                continue
            break
        return ct

In [12]:
offsets= [0.5, 0.9, 1.0]
probs =[0.5, 0.9, 1.0]

In [13]:
cats = [prob_to_cat(prob, offsets) for prob in probs]

In [14]:
cats

[0, 1, 2]

In [19]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--category_offset", type=float, nargs='+', default=[0.5, 0.9, 1.0], help="categories offset")
parser.add_argument("--other", type=float, nargs='+', default=[0.5, 0.9, 1.0], help="categories offset")
parser.parse_args("--category_offset 1 2 3 4.5 --other 4 3 2.1 5.2".split())

Namespace(category_offset=[1.0, 2.0, 3.0, 4.5], other=[4.0, 3.0, 2.1, 5.2])